#### In this notebook, assuming an effective size of 16 species, we will look at CS (BPDN,ADMM) on effectively 186 points. For each species, we will only look at predictions on the points where the species was present to begin with. (this is the second set of species that maximises data points)

#### import the relevant files and modules

In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time
from __future__ import print_function
from builtins import input


from sporco.admm import bpdn
### using the ADMM algorithm 
### we can also use the PGM algorithm 
from sporco import util
from sporco import plot
plot.config_notebook_plotting()

from scipy.linalg import hadamard

import matplotlib.pyplot as plt

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

import time 
from scipy import stats
from matplotlib import pyplot
plt.style.use('ggplot')
plt.style.use('seaborn-dark-palette')
iHiV = pd.read_pickle("~/bge-analysis-simv3/iHiV.pkl")
B = pd.read_pickle("~/bge-analysis-simv3/B.pkl")


import warnings
warnings.filterwarnings("ignore")

In [2]:
X = pd.read_pickle("~/bge-analysis-simv3/X16.pkl")

In [2]:
import matplotlib as mpl
from matplotlib import pyplot
import matplotlib.pyplot as plt
mpl.rcParams['font.family'] = 'sans-serif'
def set_violin_color(vp,color,mediancolor):
    plt.setp(vp['bodies'], facecolor=color, edgecolor="white",alpha=0.7)
    plt.setp(vp['cbars'], color = color)
    plt.setp(vp['cmins'], color = "black")
    plt.setp(vp['cmaxes'], color = "black")
    plt.setp(vp['cmeans'], color = "black")
    plt.setp(vp['cmedians'],color = mediancolor)

def make_violin_plot(dataframe,col0,mediancolor,legendlabel,diff):
    stepepi_master0 = dataframe
    bp0 = plt.violinplot(stepepi_master0, positions=np.array(range(len(stepepi_master0)))*3.0+diff,showmeans = True,showmedians=True)
    positions=np.array(range(len(stepepi_master0)))
    set_violin_color(bp0,col0,mediancolor)
    plt.plot([], c=col0, label=legendlabel)
    plt.legend(loc='upper left') 
    return positions

import seaborn as sns
colorlist1 = sns.color_palette("bright").as_hex()
sns.color_palette("bright")

from sklearn.metrics import r2_score

In [3]:
opt = bpdn.BPDN.Options({'Verbose': False, 'MaxMainIter': 500,
                         'RelStopTol': 1e-8, 'AutoRho': {'RsdlTarget': 1.0}})

#### read the effective species presence-absence

In [4]:
pa_redv1 = pd.read_pickle("~/compressed_sensingv1/realdatasets/ophelli-ryan/eff-16-list1-red-pa.pkl")

##### compile the well numbers for each species for its presence 

In [5]:
species_ones = []
for i in range(16):
    species_ones.append(pa_redv1[pa_redv1[i]==1.0]["well number"].values)

In [6]:
ones_len = [] 
for ii in range(len(species_ones)):
    ones_len.append(len(species_ones[ii])) 

#### read the steady states (relative abundances)

In [10]:
steadystate = pd.read_pickle("~/compressed_sensingv1/realdatasets/ophelli-ryan/eff-16-list1-red-sst-REL.pkl")
sst = steadystate.T

In [11]:
labdl = np.round(np.logspace(-5,0.8,5),5)
lam = list(labdl)
lam.append([0.5,0.6,1.0])
lamf = list(pd.DataFrame(lam)[0].explode().values)
lamf.sort()
lamf = lamf[:7]
lamf

[1e-05, 0.00028, 0.00794, 0.22387, 0.5, 0.6, 1.0]

In [12]:
samp = 100
n = 2**16
cv_splitsl = [2,3,5,6,7]
random_state = None
laml = lamf

In [ ]:
err = [] 
errlin  = [] 
for cv_splits in [2,3,5,6,7]:
    for species in range(2):
            m = int(samp*(ones_len[species])/100)
            cvdata = m
            ri = random.sample(list(species_ones[species]),m)
            ri.sort() 
            startt = time.time()
            y2 = sst[ri].T[species]
            data_present = y2[ri].T.index
            xs = list(data_present)
            random.shuffle(xs)
            data_present = np.array(xs)
            kf = KFold(n_splits=cv_splits)
            kf.get_n_splits(data_present)
            KFold(n_splits=cv_splits, random_state=random_state, shuffle=False)
            for lmda in laml:
                for train_index, test_index in kf.split(data_present):
                    X_train, X_test = data_present[train_index], data_present[test_index]
                    rinew = list(X_train)
                    rileft = list(X_test)
                    D = iHiV[rinew,:]
                    y3 = y2[rinew]
                    s2 = np.array([y3.values])
                    s3 = s2.T
                    b = bpdn.BPDN(D, s3, lmda, opt)
                    x = b.solve()
                    yrecon = np.dot(iHiV,x.ravel())
                    err.append([lmda,species,yrecon[rileft],rileft,rinew,cv_splits,len(rileft),len(rinew)])
                    D = X[rinew,:]
                    y3 = y2[rinew]
                    s2 = np.array([y3.values])
                    s3 = s2.T
                    b = bpdn.BPDN(D, s3, lmda, opt)
                    x = b.solve()
                    yrecon = np.dot(X,x.ravel())
                    errlin.append([lmda,species,yrecon[rileft],rileft,rinew,cv_splits,len(rileft),len(rinew)])

In [ ]:
errdf  = pd.DataFrame(err)
errdflin = pd.DataFrame(errlin)

In [ ]:
pd.to_pickle(errdf,"16l1-02-bge.pkl")

In [ ]:
pd.to_pickle(errdflin,"16l1-02-lin.pkl")

In [16]:
import pandas as pd

In [17]:
pd.read_pickle("16l1-02-bge.pkl")

0  1                                                  2  \
0    0.00001  0  [0.018920528559404556, 0.12145244076418149, 0....   
1    0.00001  0  [0.5555122248750357, -0.12824066375781445, 0.4...   
2    0.00028  0  [0.014432535711798852, 0.11739463131013483, 0....   
3    0.00028  0  [0.5285346764075642, -0.13642837017258663, 0.4...   
4    0.00794  0  [0.08144199776368491, 0.16754181869151966, 0.3...   
..       ... ..                                                ...   
317  1.00000  1  [0.3944158989863414, 0.534330417957322, 0.3944...   
318  1.00000  1  [0.586005034546647, 0.42041852831414994, 0.420...   
319  1.00000  1  [0.4100541135375763, 0.4100541135375763, 0.410...   
320  1.00000  1           [0.5300315866019827, 0.5739047230477579]   
321  1.00000  1         [0.40653072562540876, 0.40653072562540876]   

                                                     3  \
0    [41093, 36996, 33028, 32769, 45188, 37024, 450...   
1    [32770, 57349, 32900, 32788, 32772, 36870, 409...   
2    [41093, 36996, 33028, 32769, 45188, 37024, 450...   
3    [32770, 57349, 32900, 32788, 32772, 36870, 409...   
4    [41093, 36996, 33028, 32769, 45188, 37024, 450...   
..                                                 ...   
317                              [25217, 16545, 24707]   
318                              [16416, 28675, 25089]   
319                              [16512, 16384, 16386]   
320                                     [17057, 49412]   
321                                     [24579, 57349]   

                                                     4  5   6   7  
0    [32770, 57349, 32900, 32788, 32772, 36870, 409...  2  11  10  
1    [41093, 36996, 33028, 32769, 45188, 37024, 450...  2  10  11  
2    [32770, 57349, 32900, 32788, 32772, 36870, 409...  2  11  10  
3    [41093, 36996, 33028, 32769, 45188, 37024, 450...  2  10  11  
4    [32770, 57349, 32900, 32788, 32772, 36870, 409...  2  11  10  
..                                                 ... ..  ..  ..  
317  [16929, 25345, 16385, 25121, 16388, 16515, 164...  7   3  16  
318  [16929, 25345, 16385, 25121, 16388, 16515, 252...  7   3  16  
319  [16929, 25345, 16385, 25121, 16388, 16515, 252...  7   3  16  
320  [16929, 25345, 16385, 25121, 16388, 16515, 252...  7   2  17  
321  [16929, 25345, 16385, 25121, 16388, 16515, 252...  7   2  17  

[322 rows x 8 columns]